<a href="https://colab.research.google.com/github/ChenYongyan-uu/1/blob/master/3_Herd_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.co

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType
import pyspark
from pyspark.sql import functions as F,Window
from pyspark.sql.functions import year, col, array, monotonically_increasing_id, to_json, struct, round, datediff,unix_timestamp,udf,avg
import json
from urllib.request import  urlopen
import pyspark.sql.functions as F

## Load in fitting result

In [ ]:
FittingPerLactationFiltered = spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/FittingPerLactationFiltered')

In [ ]:
FittingPerLactationFiltered.printSchema()

root
 |-- HerdIdentifier: integer (nullable = true)
 |-- AnimalIdentifier: string (nullable = true)
 |-- CalvingDate: date (nullable = true)
 |-- Parity: double (nullable = true)
 |-- DaysInMilk: double (nullable = true)
 |-- Points: double (nullable = true)
 |-- ScaleKg: double (nullable = true)
 |-- Ramp: double (nullable = true)
 |-- Offset: double (nullable = true)
 |-- Decay: double (nullable = true)
 |-- Persistence: double (nullable = true)
 |-- StdErrKg: double (nullable = true)
 |-- M305Kg: double (nullable = true)
 |-- ParityGroup: string (nullable = true)



# Herd summary

## CalvingYear

In [ ]:
df1=FittingPerLactationFiltered.withColumn('CalvingYear',year('CalvingDate').cast(IntegerType()))

In [ ]:
df1.show(2)

+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|Parity|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|CalvingYear|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----------+
|           590|   NL 3233658025| 2008-07-31|   6.0|     377.0|   7.0|  52.76|34.8|   0.0|0.00233|      297.0|     1.9|10644.0|         2+|       2008|
|           590|   NL 3514206103| 2006-12-28|   3.0|     476.0|   8.0|  52.58| 8.5|   0.0|0.00219|      316.0|     1.8|11454.0|         2+|       2006|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----------+
only showing top 2 rows



In [ ]:
from pyspark.sql.functions import countDistinct, avg

df1=df1.groupBy('HerdIdentifier','ParityGroup','CalvingYear').\
  agg(avg('ScaleKg').alias('MeanScaleKg'),
  avg('Ramp').alias('MeanRamp'),
  avg('Offset').alias('MeanOffset'),
  avg('Decay').alias('MeanDecay'),
  avg('Persistence').alias('MeanPersistence'),
  avg('M305Kg').alias('MeanM305Kg'),
  F.expr('percentile(ScaleKg, array(0.5))')[0].alias('MedianScaleKg'),
  F.expr('percentile(Ramp,array(0.5))')[0].alias('MedianRamp'),
  F.expr('percentile(Offset,array(0.5))')[0].alias('MedianOffset'),
  F.expr('percentile(Decay,array(0.5))')[0].alias('MedianDecay'),
  F.expr('percentile(Persistence,array(0.5))')[0].alias('MedianPersistence'),
  F.expr('percentile(M305Kg,array(0.5))')[0].alias('MedianM305Kg'),
  F.count(F.lit(1)).alias("NumberOfAnimal")
  )

In [ ]:
df1.count()

437741

In [ ]:
df1.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/CalvingYear',mode="overwrite")

## EndYear

In [ ]:
FittingPerLactationFiltered=FittingPerLactationFiltered.withColumn('LastDate',F.expr('date_add(CalvingDate,int(DaysInMilk))'))

In [ ]:
FittingPerLactationFiltered.show(2)

+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+----------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|Parity|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|  LastDate|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+----------+
|           590|   NL 3233658025| 2008-07-31|   6.0|     377.0|   7.0|  52.76|34.8|   0.0|0.00233|      297.0|     1.9|10644.0|         2+|2009-08-12|
|           590|   NL 3514206103| 2006-12-28|   3.0|     476.0|   8.0|  52.58| 8.5|   0.0|0.00219|      316.0|     1.8|11454.0|         2+|2008-04-17|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+----------+
only showing top 2 rows



In [ ]:
df2=FittingPerLactationFiltered.withColumn('EndYear',year('LastDate').cast(IntegerType()))

In [ ]:
from pyspark.sql.functions import countDistinct, avg

df2=df2.groupBy('HerdIdentifier','ParityGroup','EndYear').\
  agg(avg('ScaleKg').alias('MeanScaleKg'),
  avg('Ramp').alias('MeanRamp'),
  avg('Offset').alias('MeanOffset'),
  avg('Decay').alias('MeanDecay'),
  avg('Persistence').alias('MeanPersistence'),
  avg('M305Kg').alias('MeanM305Kg'),
  F.expr('percentile(ScaleKg, array(0.5))')[0].alias('MedianScaleKg'),
  F.expr('percentile(Ramp,array(0.5))')[0].alias('MedianRamp'),
  F.expr('percentile(Offset,array(0.5))')[0].alias('MedianOffset'),
  F.expr('percentile(Decay,array(0.5))')[0].alias('MedianDecay'),
  F.expr('percentile(Persistence,array(0.5))')[0].alias('MedianPersistence'),
  F.expr('percentile(M305Kg,array(0.5))')[0].alias('MedianM305Kg'),
  F.count(F.lit(1)).alias("NumberOfAnimal")
  )

In [ ]:
df2.count()

380309

In [ ]:
df2.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/EndYear',mode="overwrite")

## Weighted mean & median

### calculate weight


#### load in filtered test-day data

In [ ]:
spark_alldata_filtered = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/spark_alldata_filtered/*') \

In [ ]:
spark_alldata_filtered.show(3)

+----------------+------+-----------+--------------+--------+----------+------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+---------+------------------+---------------+
|AnimalIdentifier|Parity|CalvingDate|HerdIdentifier|TestYear| BirthDate|BusinessType|      Date|MilkYieldKg|FatPercentage|ProteinPercentage|     SCC|Urea|LactosePercentage|NumberOfMilkings|StatusProduction|StatusFictive|StatusCow|LactationValue|DaysInMilk|AgeInDays|              FPCM|CalvingInterval|
+----------------+------+-----------+--------------+--------+----------+------------+----------+-----------+-------------+-----------------+--------+----+-----------------+----------------+----------------+-------------+---------+--------------+----------+---------+------------------+---------------+
|   AT 1796270067|   4.0| 2017-01-01|       9208246|    2017|2011-03-15|         UBN|2017-01-1

In [ ]:
spark_alldata_filtered.count()

130939857

In [ ]:
spark_alldata_filtered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct('AnimalIdentifier')).collect()

[Row(count(DISTINCT HerdIdentifier)=17580, count(DISTINCT AnimalIdentifier)=5894719)]

#### get final weight

In [ ]:
weight=spark_alldata_filtered.groupBy('HerdIdentifier','AnimalIdentifier','CalvingDate','Parity','TestYear').\
  agg(F.count('*').alias('Weight'))

In [ ]:
weight.show(3)

+--------------+----------------+-----------+------+--------+------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|Parity|TestYear|weight|
+--------------+----------------+-----------+------+--------+------+
|       5000959|   CZ 9811962638| 2018-08-06|   4.0|    2018|     3|
|        606467|   DE 0947017669| 2012-07-26|   2.0|    2012|     3|
|       1316903|   DE 1302028801| 2018-01-07|   5.0|    2018|     3|
+--------------+----------------+-----------+------+--------+------+
only showing top 3 rows



In [ ]:
weight.agg(F.countDistinct("HerdIdentifier"),F.countDistinct('AnimalIdentifier')).collect()

[Row(count(HerdIdentifier)=17580, count(AnimalIdentifier)=5894719)]

### combine weight with FittingResult

In [ ]:
FittingPerLactationFiltered = spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/FittingPerLactationFiltered')

In [ ]:
FittingPerLactationFiltered.show(3)

+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|Parity|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
|           590|   NL 3233658025| 2008-07-31|   6.0|     377.0|   7.0|  52.76|34.8|   0.0|0.00233|      297.0|     1.9|10644.0|         2+|
|           590|   NL 3514206103| 2006-12-28|   3.0|     476.0|   8.0|  52.58| 8.5|   0.0|0.00219|      316.0|     1.8|11454.0|         2+|
|           590|   NL 4146636720| 2006-09-14|   1.0|     406.0|   6.0|  44.55|30.0|  -0.5|0.00184|      378.0|     1.9| 9759.0|          1|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
only showing top 3 r

In [ ]:
FittingPerLactationFiltered.count()

15726779

In [ ]:
FittingPerLactationFiltered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct('AnimalIdentifier')).collect()

[Row(count(HerdIdentifier)=17579, count(AnimalIdentifier)=5773825)]

In [ ]:
Weight=weight.join(FittingPerLactationFiltered,['HerdIdentifier','AnimalIdentifier','Parity','CalvingDate'],'inner')

In [ ]:
Weight.count()

25824630

In [ ]:
Weight.orderBy('HerdIdentifier','AnimalIdentifier','Parity').show(10)

+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
|HerdIdentifier|AnimalIdentifier|Parity|CalvingDate|TestYear|Weight|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|
+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
|           590|   NL 1838167674|   7.0| 2006-08-04|    2007|     3|     273.0|   3.0|   48.4|22.5|   0.0|0.00281|      247.0|     2.1| 9386.0|         2+|
|           590|   NL 2045406648|   6.0| 2006-01-21|    2007|     2|     418.0|   2.0|  48.65|22.5|   0.0| 0.0023|      302.0|     4.6|10121.0|         2+|
|           590|   NL 2045406648|   7.0| 2007-05-19|    2008|     6|     494.0|  10.0|  37.64|33.1|   0.0|0.00119|      583.0|     2.6| 9003.0|         2+|
|           590|   NL 2045406648|   7.0| 2007-05-19|    2007|   

In [ ]:
from pyspark.sql import Window
w=Window.partitionBy('HerdIdentifier','AnimalIdentifier','Parity')

In [ ]:
Weight=Weight.withColumn('NYear',F.count('*').over(w)).\   # how many calendar years
      withColumn('ParityGroup',F.when(F.col('Parity')==1,'1').otherwise('2+'))

In [ ]:
Weight.show(5)

+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----+
|HerdIdentifier|AnimalIdentifier|Parity|CalvingDate|TestYear|Weight|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|NYear|
+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----+
|           590|   NL 2343569704|   6.0| 2006-04-13|    2007|     2|     336.0|   2.0|  48.16|22.5|   0.0|0.00284|      244.0|     1.6| 9295.0|         2+|    1|
|           590|   NL 3696560646|   6.0| 2012-03-05|    2013|     3|     463.0|   9.0|  56.28|30.9|   0.0|0.00306|      227.0|     2.6|10347.0|         2+|    2|
|           590|   NL 3696560646|   6.0| 2012-03-05|    2012|     6|     463.0|   9.0|  56.28|30.9|   0.0|0.00306|      227.0|     2.6|10347.0|         2+|    2|
|           590|   NL 372564

In [ ]:
Weight.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/Weight',mode="overwrite")

In [ ]:
Weight= spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/Weight')

In [ ]:
Weight.groupBy('NYear').\
  agg(F.countDistinct('HerdIdentifier','AnimalIdentifier','Parity').alias('NumOfLactation')).orderBy('NYear').show()

+-----+--------------+
|NYear|NumOfLactation|
+-----+--------------+
|    1|       5809488|
|    2|       9612036|
|    3|        263690|
+-----+--------------+



### Calculate weighted mean&median

In [ ]:
n_to_array = udf(lambda n : [n] * n, ArrayType(IntegerType()))

In [ ]:
df=Weight.withColumn('WeightArray',n_to_array(Weight.Weight))

In [ ]:
df.show(3)

+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----+------------------+
|HerdIdentifier|AnimalIdentifier|Parity|CalvingDate|TestYear|Weight|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|NYear|       WeightArray|
+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----+------------------+
|           590|   NL 2343569704|   6.0| 2006-04-13|    2007|     2|     336.0|   2.0|  48.16|22.5|   0.0|0.00284|      244.0|     1.6| 9295.0|         2+|    1|            [2, 2]|
|           590|   NL 3696560646|   6.0| 2012-03-05|    2013|     3|     463.0|   9.0|  56.28|30.9|   0.0|0.00306|      227.0|     2.6|10347.0|         2+|    2|         [3, 3, 3]|
|           590|   NL 3696560646|   6.0| 2012-03-05|    2012|     6|     463.0|   9.0|  56.28|3

In [ ]:
WeightedExplore=df.withColumn('weight',F.explode(df.WeightArray))   #duplicate the lactation curve characteristics according to their weights

In [ ]:
WeightedExplore.show(10)

+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----+------------------+
|HerdIdentifier|AnimalIdentifier|Parity|CalvingDate|TestYear|weight|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|NYear|       WeightArray|
+--------------+----------------+------+-----------+--------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+-----+------------------+
|           590|   NL 2343569704|   6.0| 2006-04-13|    2007|     2|     336.0|   2.0|  48.16|22.5|   0.0|0.00284|      244.0|     1.6| 9295.0|         2+|    1|            [2, 2]|
|           590|   NL 2343569704|   6.0| 2006-04-13|    2007|     2|     336.0|   2.0|  48.16|22.5|   0.0|0.00284|      244.0|     1.6| 9295.0|         2+|    1|            [2, 2]|
|           590|   NL 3696560646|   6.0| 2012-03-05|    2013|     3|     463.0|   9.0|  56.28|3

#### weighted mean

In [ ]:
WeightedMean=WeightedExplore.groupBy('HerdIdentifier','TestYear','ParityGroup').\
  agg(avg('ScaleKg').alias('MeanScaleKg'),
  avg('Ramp').alias('MeanRamp'),
  avg('Offset').alias('MeanOffset'),
  avg('Decay').alias('MeanDecay'),
  avg('Persistence').alias('MeanPersistence'),
  avg('M305Kg').alias('MeanM305Kg'),
  F.countDistinct('AnimalIdentifier').alias('NumberOfAnimal'),
  F.count(F.lit(1)).alias('TotalWeight')
  )

In [ ]:
WeightedMean.show(10)

+--------------+--------+-----------+------------------+------------------+--------------------+--------------------+------------------+------------------+--------------+-----------+
|HerdIdentifier|TestYear|ParityGroup|       MeanScaleKg|          MeanRamp|          MeanOffset|           MeanDecay|   MeanPersistence|        MeanM305Kg|NumberOfAnimal|TotalWeight|
+--------------+--------+-----------+------------------+------------------+--------------------+--------------------+------------------+------------------+--------------+-----------+
|        311983|    2017|         2+| 55.46420924574209|25.840551500405518|-0.29261962692619636|0.002771730489613701|282.17761557177613|10747.456153432668|           263|       1233|
|        421225|    2016|         2+| 47.24128404669262|20.361867704280154|-0.21478599221789885|0.002571789883268483| 315.3774319066148| 9555.515564202335|           102|        514|
|        583082|    2009|         2+| 39.11147260273972| 17.89178082191781|-0.3287671

In [ ]:
WeightedMean.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/WeightedMean',mode="overwrite")

#### weighted median

In [ ]:
WeightedMedian=WeightedExplore.groupBy('HerdIdentifier','TestYear','ParityGroup').\
  agg(F.expr('percentile(ScaleKg, array(0.5))')[0].alias('MedianScaleKg'),
  F.expr('percentile(Ramp,array(0.5))')[0].alias('MedianRamp'),
  F.expr('percentile(Offset,array(0.5))')[0].alias('MedianOffset'),
  F.expr('percentile(Decay,array(0.5))')[0].alias('MedianDecay'),
  F.expr('percentile(Persistence,array(0.5))')[0].alias('MedianPersistence'),
  F.expr('percentile(M305Kg,array(0.5))')[0].alias('MedianM305Kg'),
  )

In [ ]:
WeightedMedian.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/WeightedMedian',mode="overwrite")

### Combine mean and median result 

In [ ]:
WeightedMedian= spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/WeightedMedian')

In [ ]:
WeightedMedian.show(3)

+--------------+--------+-----------+-------------+----------+------------+-----------+-----------------+------------+
|HerdIdentifier|TestYear|ParityGroup|MedianScaleKg|MedianRamp|MedianOffset|MedianDecay|MedianPersistence|MedianM305Kg|
+--------------+--------+-----------+-------------+----------+------------+-----------+-----------------+------------+
|           919|    2019|         2+|        53.61|      22.2|        -0.8|    0.00299|            232.0|     10162.0|
|          3388|    2013|         2+|        47.91|      22.1|         0.0|    0.00238|            291.0|      9555.0|
|          4909|    2008|          1|        35.27|      29.6|        -0.5|    0.00223|            311.0|      7391.0|
+--------------+--------+-----------+-------------+----------+------------+-----------+-----------------+------------+
only showing top 3 rows



In [ ]:
WeightedMedian.count()

380607

In [ ]:
WeightedMean= spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/WeightedMean')

In [ ]:
WeightedMean.show(3)

+--------------+--------+-----------+-----------------+------------------+--------------------+--------------------+------------------+------------------+--------------+-----------+
|HerdIdentifier|TestYear|ParityGroup|      MeanScaleKg|          MeanRamp|          MeanOffset|           MeanDecay|   MeanPersistence|        MeanM305Kg|NumberOfAnimal|TotalWeight|
+--------------+--------+-----------+-----------------+------------------+--------------------+--------------------+------------------+------------------+--------------+-----------+
|        311983|    2017|         2+|55.46420924574209|25.840551500405518|-0.29261962692619636|0.002771730489613701|282.17761557177613|10747.456153432668|           263|       1233|
|        421225|    2016|         2+|47.24128404669262|20.361867704280154|-0.21478599221789885|0.002571789883268483| 315.3774319066148| 9555.515564202335|           102|        514|
|        583082|    2009|         2+|39.11147260273972| 17.89178082191781|-0.3287671232876

In [ ]:
WeightedMean.count()

380607

In [ ]:
Weighted=WeightedMean.join(WeightedMedian,['HerdIdentifier','TestYear','ParityGroup'],'inner')

In [ ]:
Weighted.count()

380607

In [ ]:
Weighted.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/Weighted',mode="overwrite")